In [26]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import boto3
import sagemaker 
from sagemaker import get_execution_role

In [27]:
import boto3
forecast = boto3.client('forecast')

In [28]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install boto3[crt]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [30]:
import boto3
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

sqs = boto3.resource('sqs')


ethpricedataset


In [55]:
import boto3
client = boto3.client('forecast')
ds_response = client.create_dataset(
    DatasetName = 'geminidataset',
    Domain='CUSTOM',
    DatasetType = 'TARGET_TIME_SERIES',
    DataFrequency ='D',
    Schema={
        'Attributes':[
            {
                'AttributeName': 'item_id',
                'AttributeType': 'string'
            },
            {
                'AttributeName': 'timestamp',
                'AttributeType': 'timestamp'
            },
            {
                'AttributeName':'target_value',
                'AttributeType':'float'
            },
        ]
    }
)

In [56]:
dataset_arn= ds_response['DatasetArn']
print(dataset_arn)

arn:aws:forecast:ap-south-1:950788233543:dataset/geminidataset


In [58]:
dsgrp_response = client.create_dataset_group(
    DatasetGroupName='forecastgrp1',
    Domain='CUSTOM',
    DatasetArns=[
        dataset_arn,
    ],
)

In [59]:
print(dsgrp_response)

{'DatasetGroupArn': 'arn:aws:forecast:ap-south-1:950788233543:dataset-group/forecastgrp1', 'ResponseMetadata': {'RequestId': '2f50f345-b469-483c-aa10-59784a4b5070', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 09 Apr 2023 17:00:53 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '89', 'connection': 'keep-alive', 'x-amzn-requestid': '2f50f345-b469-483c-aa10-59784a4b5070'}, 'RetryAttempts': 0}}


In [66]:
dsgrp_arn='arn:aws:forecast:ap-south-1:950788233543:dataset-group/forecastgrp1'

In [69]:
forecastHorizon=24
create_predictor_response=client.create_predictor(PredictorName="forecastethpredic", 
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML=True,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": dsgrp_arn},
                                                  FeaturizationConfig= {"ForecastFrequency": "D", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [70]:
predictorArn=create_predictor_response['PredictorArn']
print(predictorArn)

arn:aws:forecast:ap-south-1:950788233543:predictor/forecastethpredic


In [71]:
predictorArn=create_predictor_response['PredictorArn']
in_progress_predictors = [
     create_predictor_response['PredictorArn']
]
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for predictor_arn in in_progress_predictors:
        predictor_response = client.describe_predictor(
            PredictorArn = predictorArn
        )
        status = predictor_response["Status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(predictorArn))
            in_progress_predictors.remove(predictorArn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(predictorArn))
            in_progress_predictors.remove(predictorArn)
    
    if len(in_progress_predictors) <= 0:
        break
    else:
        print("At least one Predictor build is still in progress")
        
    time.sleep(60)

At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress


In [72]:
client.get_accuracy_metrics(PredictorArn=predictorArn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 1836.0346906138257,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.033736219007727386},
       {'Quantile': 0.5, 'LossValue': 0.03487298731093453},
       {'Quantile': 0.1, 'LossValue': 0.021560980891647232}],
      'ErrorMetrics': [{'ForecastType': 'mean',
        'WAPE': 0.0376606451532956,
        'RMSE': 1836.0346906138257,
        'MASE': 2.3596044528574764,
        'MAPE': 0.03849646385247681}],
      'AverageWeightedQuantileLoss': 0.03005672907010305}},
    {'TestWindowStart': datetime.datetime(2023, 2, 10, 23, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2023, 3, 6, 23, 0, tzinfo=tzlocal()),
     'ItemCount': 1,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 1836.0346906138257,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.033736

In [73]:
forecastName="cvvprojforecast"
create_forecast_response=client.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']
print(forecastArn)

arn:aws:forecast:ap-south-1:950788233543:forecast/cvvprojforecast


In [74]:
in_progress_forecast = [
    create_forecast_response['ForecastArn']
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for forecast_arn in in_progress_forecast:
        forecast_response = forecast.describe_forecast(
            ForecastArn = forecast_arn
        )
        status = forecast_response["Status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(forecast_arn))
            in_progress_forecast.remove(forecast_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(forecast_arn))
            in_progress_forecast.remove(forecast_arn)
    
    if len(in_progress_forecast) <= 0:
        break
    else:
        print("At least one Forecast build is still in progress")
        
    time.sleep(60)

At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
At least one Forecast build is still in progress
Build succeeded for arn:aws:forecast:ap-south-1:950788233543:forecast/cvvprojforecast


In [75]:
forecastExportName='Export_forecasts'
outputPath="s3://ethpricedataset/Gemini_ETHforecastresults"
forecast_export_response = forecast.create_forecast_export_job(
                                                                ForecastExportJobName = forecastExportName,
                                                                ForecastArn=forecastArn, 
                                                                Destination = {
                                                                   "S3Config" : {
                                                                       "Path":outputPath,
                                                                       "RoleArn": role_arn
                                                                   } 
                                                                }
                                                              )

In [76]:
forecastExportJobArn = forecast_export_response['ForecastExportJobArn']
print(forecastExportJobArn)

arn:aws:forecast:ap-south-1:950788233543:forecast-export-job/cvvprojforecast/Export_forecasts
